In [ ]:
import chardet
import pickle
import pandas as pd
import numpy as np

from sklearn.impute import IterativeImputer
from sklearn.experimental import enable_iterative_imputer

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression



Vérifie l'encodage

In [ ]:
with open("data/houses_madrid_eda_features.csv", "rb") as fichier_brut:
    encodage = chardet.detect(fichier_brut.read(1000))
print(encodage)

{'encoding': 'ascii', 'confidence': 1.0, 'language': ''}


Importe le csv

In [14]:
df = pd.read_csv("data/houses_madrid_eda_features.csv", encoding= 'ascii')
df.head(1)

,log_buy_price,rent_price,buy_price,n_bathrooms,sq_mt_built,price_per_m2,n_rooms,has_lift,has_parking,has_garden,has_storage_room,has_central_heating,has_pool
0,11.350418,471.0,85000,1.0,64.0,1328.125,2,0.0,0,0,0,NaN,0


Exporte et réimporte en .feather (plus rapide que .csv)

In [18]:
df.to_feather(r'data/houses.feather')
df = pd.read_feather("data/houses.feather")
df.head(1)

,log_buy_price,rent_price,buy_price,n_bathrooms,sq_mt_built,price_per_m2,n_rooms,has_lift,has_parking,has_garden,has_storage_room,has_central_heating,has_pool
0,11.350418,471.0,85000,1.0,64.0,1328.125,2,0.0,0,0,0,NaN,0


Variables vides

In [21]:
empty_variables = df.columns[df.isna().all()].tolist()
empty_variables

[]

Taux de remplissage des variables

In [23]:
fill_rate = df.count() / len(df) * 100
fill_rate

log_buy_price          100.000000
rent_price              87.825407
buy_price              100.000000
n_bathrooms             99.926410
sq_mt_built             99.420476
price_per_m2            99.420476
n_rooms                100.000000
has_lift                89.025849
has_parking            100.000000
has_garden             100.000000
has_storage_room       100.000000
has_central_heating     62.588538
has_pool               100.000000
dtype: float64

In [29]:
# Imputation par régression itérative (IterativeImputer)
imputer = IterativeImputer(random_state=42, max_iter=10)
df_imputed = pd.DataFrame(imputer.fit_transform(df), columns=df.columns)
df = df_imputed
print("✅ Imputation par régression complétée")
print(f"Valeurs manquantes restantes: {df.isna().sum().sum()}")

✅ Imputation par régression complétée
Valeurs manquantes restantes: 0


In [31]:
df.shape

(21742, 13)

In [ ]:
# # Imputation par la moyenne avec le mode
# df['variable'] = df['Variable'].fillna(df['Variable'].mode()[0])

# # Imputation par la moyenne avec pandas
# df['variable'] = df['Variable'].fillna(df['Variable'].mean())

# # Imputation par la mediane avec pandas
# df['variable'] = df['Variable'].fillna(df['Variable'].median())

Séparation variables explicatives et déopendante
Split train et Test

In [ ]:
X = df.drop(columns=['log_buy_price'])
y = df['log_buy_price']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = LinearRegression()
model.fit(X, y)

# Sauvegarder
with open("model.pkl", "wb") as f:
    pickle.dump(model, f)

print("✅ Modèle créé : model.pkl")

✅ Modèle factice créé : model.pkl


TypeError: RegressorMixin.score() missing 2 required positional arguments: 'X' and 'y'